# Topic modelling of news headlines for prediction of news category¶

Now we wan't to perform topic modelling with unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier on the combined feature 'title_description_text' feature and assume 7 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

## Load data

In [2]:
train = pd.read_csv('../../data/01_train_nosplit_preprocessed.csv')

In [3]:
train.head()

,date,title,description,category,text,source,title_description_text
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...


## Load German stopwords

In [5]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [6]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

## Extract features from 'title_description_text' 

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
cv = CountVectorizer(max_df=0.95, min_df=3, stop_words=stop_words)

In [9]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['title_description_text'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [10]:
dtm

<67513x37203 sparse matrix of type '<class 'numpy.int64'>'
	with 1119124 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.decomposition import LatentDirichletAllocation

In [12]:
lda = LatentDirichletAllocation(n_components=7, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

## Analyse extracted features

In [13]:
len(cv.get_feature_names_out())

37203

In [14]:
cv.get_feature_names_out()

array(['00', '000', '007', ..., 'но', 'украинцы', 'это'], dtype=object)

In [15]:
len(lda.components_)

7

In [16]:
lda.components_

array([[1.42857287e-01, 1.71782364e+02, 1.42857206e-01, ...,
        1.42857255e-01, 1.42857266e-01, 1.42857217e-01],
       [1.43049990e-01, 4.79974164e+02, 4.03203084e+00, ...,
        1.42857314e-01, 1.42857330e-01, 1.42857256e-01],
       [1.13470320e+00, 2.21429581e+02, 1.43962034e-01, ...,
        1.42857324e-01, 1.42857341e-01, 1.42857262e-01],
       ...,
       [1.49367614e-01, 2.74996298e+02, 2.50906061e-01, ...,
        1.42857320e-01, 1.42857337e-01, 1.42857260e-01],
       [1.43459929e-01, 1.43094191e-01, 1.43528435e-01, ...,
        1.42857308e-01, 1.42857324e-01, 1.42857252e-01],
       [1.43255991e-01, 1.88541924e+02, 1.42857246e-01, ...,
        4.14285614e+00, 5.14285604e+00, 4.14285648e+00]])

In [17]:
len(lda.components_[0])

37203

### Show most important words of first extracted topic

In [18]:
first_topic = lda.components_[0]

In [19]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([35080,  9604,  9208, ..., 26228, 18192, 31472], dtype=int64)

In [20]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [21]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

kiew
ukrainische
russischen
china
scholz
russische
putin
russland
krieg
ukraine


### Show most important words of first extracted topic

In [22]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['moskau', 'usa', 'russlands', 'präsident', 'selenskyj', 'kiew', 'ukrainische', 'russischen', 'china', 'scholz', 'russische', 'putin', 'russland', 'krieg', 'ukraine']


Die TOP-15 Wörter für das Thema #1
['geld', 'länder', 'millionen', 'deutsche', 'neue', 'milliarden', 'faeser', 'gibt', 'menschen', 'inflation', 'prozent', 'jahr', 'euro', 'deutschland', 'eu']


Die TOP-15 Wörter für das Thema #2
['em', 'deutsche', 'deutschen', 'bahn', 'focus', 'wegen', 'online', 'frankreich', 'menschen', 'news', 'lauterbach', 'pandemie', 'deutschland', 'wm', 'corona']


Die TOP-15 Wörter für das Thema #3
['sieg', 'münchen', 'letzte', 'berliner', 'gestorben', 'ex', 'fußball', 'league', 'generation', 'bundesliga', 'fc', 'jahren', 'iran', 'bayern', 'berlin']


Die TOP-15 Wörter für das Thema #4
['boris', 'klima', 'eu', 'euro', 'menschen', 'energie', 'ampel', 'deutschen', 'neue', 'regierung', 'lindner', 'bundesregierung', 'gas', 'habeck', 'deutschland']


Die TOP-15 Wörter

### Get topics with highest probability for news items in train data

In [23]:
topic_results = lda.transform(dtm)
topic_results.shape

(67513, 7)

In [24]:
topic_results[0].round(5)

array([0.00622, 0.22813, 0.00622, 0.00623, 0.74073, 0.00625, 0.00622])

In [25]:
topic_results[0].argmax()

4

In [26]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [27]:
train.head()

,date,title,description,category,text,source,title_description_text,topic
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,4
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,0
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,5
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,0
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,1


## Save train set with topics and dump model

In [28]:
#Save result to csv
train.to_csv('train_lda_combined.csv')

In [29]:
# Dump LDA model
joblib.dump(lda, 'lda_model_7_topics_combined.jl')

['lda_model_7_topics_combined.jl']